In [1]:
import numpy as np
from collections import Counter

In [2]:
# 用于打印数据
def print_var(var, name='x'):
    print(name+':\n',var, '\nshape_or_type:', var.shape if hasattr(var, 'shape') else type(var))

In [3]:
#导入例4.1中的数据
data = np.array([
    [1, 'S', -1],
    [1, 'M', -1], 
    [1, 'M', 1],
    [1, 'S', 1],
    [1, 'S', -1],
    [2, 'S', -1],
    [2, 'M', -1],
    [2, 'M', 1],
    [2, 'L', 1],
    [2, 'L', 1],
    [3, 'L', 1],
    [3, 'M', 1],
    [3, 'M', 1],
    [3, 'L', 1],
    [3, 'L', -1],
    ])

In [4]:
X = data[:, :2]
print_var(X, 'X')
Y = data[:, 2]
print_var(Y, 'Y')

X:
 [['1' 'S']
 ['1' 'M']
 ['1' 'M']
 ['1' 'S']
 ['1' 'S']
 ['2' 'S']
 ['2' 'M']
 ['2' 'M']
 ['2' 'L']
 ['2' 'L']
 ['3' 'L']
 ['3' 'M']
 ['3' 'M']
 ['3' 'L']
 ['3' 'L']] 
shape_or_type: (15, 2)
Y:
 ['-1' '-1' '1' '1' '-1' '-1' '-1' '1' '1' '1' '1' '1' '1' '1' '-1'] 
shape_or_type: (15,)


In [5]:
# 计算每个label的先验概率
PY = {y:cnt / Y.shape[0] for y, cnt in Counter(Y).items()}
PY

{'-1': 0.4, '1': 0.6}

In [6]:
# 计算条件概率
PYX = {}
# 遍历每个label
for y_label in ['-1', '1']:
    # 取y = y_label时的数据sub_x
    sub_x = X[Y == y_label]
    # 遍历X的每个维度特征，例子中sub_x.shape[1] = 2，因为只有两个维度
    for x_dim in range(sub_x.shape[1]):
        dim_sub_x = sub_x[:, x_dim]
        #计算每个特征每个值出现的概率
        for x_v, cnt in Counter(dim_sub_x).items():
            #保存条件概率，当类别为y_label时，第x_dim特征维度的值为x_v时的概率
            PYX[(x_dim, x_v, y_label)] = cnt / dim_sub_x.shape[0]
            
PYX

{(0, '1', '-1'): 0.5,
 (0, '1', '1'): 0.2222222222222222,
 (0, '2', '-1'): 0.3333333333333333,
 (0, '2', '1'): 0.3333333333333333,
 (0, '3', '-1'): 0.16666666666666666,
 (0, '3', '1'): 0.4444444444444444,
 (1, 'L', '-1'): 0.16666666666666666,
 (1, 'L', '1'): 0.4444444444444444,
 (1, 'M', '-1'): 0.3333333333333333,
 (1, 'M', '1'): 0.4444444444444444,
 (1, 'S', '-1'): 0.5,
 (1, 'S', '1'): 0.1111111111111111}

In [7]:
test_x = ['2', 'S']

# 按照公式计算后验概率
# p(Y=1)
y_label = '1'
p_1 = PY[y_label] * PYX[(0, '2', y_label)] * PYX[(1, 'S', y_label)]

# p(Y='-1')
y_label = '-1'
p_2 = PY[y_label] * PYX[(0, '2', y_label)] * PYX[(1, 'S', y_label)]

p1 = p_1 / (p_1 + p_2)
p2 = p_2 / (p_1 + p_2)


pre_y = (p1, '1') if p1 > p2 else (p2, '-1')
pre_y

(0.7500000000000001, '-1')

### 贝叶斯估
引入一个正数lambda,用于对条件概率进行平滑

In [8]:
lamb = 1

In [9]:
# 计算每个label的先验概率
# 其中加入平滑
cnt_y = Counter(Y)
S = len(cnt_y)
PY = {y:(cnt + lamb) / (Y.shape[0] + S * lamb) for y, cnt in Counter(Y).items()}
PY

{'-1': 0.4117647058823529, '1': 0.5882352941176471}

In [10]:
# 计算条件概率

PYX = {}
# 遍历每个label
for y_label in ['-1', '1']:
    # 取y = y_label时的数据sub_x
    sub_x = X[Y == y_label]
    # 遍历X的每个维度特征，例子中sub_x.shape[1] = 2，因为只有两个维度
    for x_dim in range(sub_x.shape[1]):
        dim_sub_x = sub_x[:, x_dim]
        #计算每个特征每个值出现的概率
        cnt_dim_sub_x = Counter(dim_sub_x)
        S = len(cnt_dim_sub_x)
        for x_v, cnt in cnt_dim_sub_x.items():
            # 保存条件概率，当类别为y_label时，第x_dim特征维度的值为x_v时的概率
            # 其中加入平滑
            PYX[(x_dim, x_v, y_label)] = (cnt + lamb) / (dim_sub_x.shape[0] + S * lamb)
            
PYX

{(0, '1', '-1'): 0.4444444444444444,
 (0, '1', '1'): 0.25,
 (0, '2', '-1'): 0.3333333333333333,
 (0, '2', '1'): 0.3333333333333333,
 (0, '3', '-1'): 0.2222222222222222,
 (0, '3', '1'): 0.4166666666666667,
 (1, 'L', '-1'): 0.2222222222222222,
 (1, 'L', '1'): 0.4166666666666667,
 (1, 'M', '-1'): 0.3333333333333333,
 (1, 'M', '1'): 0.4166666666666667,
 (1, 'S', '-1'): 0.4444444444444444,
 (1, 'S', '1'): 0.16666666666666666}

In [11]:
test_x = ['2', 'S']

# 按照公式计算后验概率
# p(Y=1)
y_label = '1'
p_1 = PY[y_label] * PYX[(0, '2', y_label)] * PYX[(1, 'S', y_label)]

# p(Y='-1')
y_label = '-1'
p_2 = PY[y_label] * PYX[(0, '2', y_label)] * PYX[(1, 'S', y_label)]

p1 = p_1 / (p_1 + p_2)
p2 = p_2 / (p_1 + p_2)


pre_y = (p1, '1') if p1 > p2 else (p2, '-1')
pre_y

(0.6511627906976744, '-1')